In [3]:
import requests
import json   


def flatten_json(data, parent_key='', sep='_'):
    flattened = {}
    for key, value in data.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key
        if isinstance(value, dict):
            flattened.update(flatten_json(value, new_key, sep=sep))
        else:
            flattened[new_key] = value
    return flattened

def create_pretty_table(data, title):
    from prettytable import PrettyTable
    
    table = PrettyTable()
    table.field_names = [title, "Values"]
    for key, value in data.items():
        table.add_row([key, value])
    return table

def display_stock_data(stock_data):
    # Extract meta_data and time_series_data
    meta_data = stock_data.get('Meta Data', {})
    time_series_data = stock_data.get('Time Series (Daily)', {})

    # Flatten meta data and time series data
    flattened_meta = flatten_json(meta_data)
    flattened_time_series = {date: flatten_json(data) for date, data in time_series_data.items()}

    # Create PrettyTable for flattened meta data
    meta_table = create_pretty_table(flattened_meta, "Meta Data")

    # Create PrettyTable for flattened time series data
    time_series_table = PrettyTable()
    if flattened_time_series:
        keys = list(flattened_time_series.values())[0].keys()
        time_series_table.field_names = ["Date"] + list(keys)
        for date, data in flattened_time_series.items():
            time_series_table.add_row([date] + [data.get(key, '') for key in keys])

    # Print the tables
    print("Meta Data:")
    print(meta_table)
    print("\nTime Series (Daily):")
    print(time_series_table)

def get_stock_data(stock_name):

    # As alphavantage allows for only 25 calls/day we will try different keys
    api_keys ={}
    api_keys['key1'] = '49MPJB381SMB9GL2'
    api_keys['key2'] = 'R74FDR76XE38N28Q'
    api_keys['key3'] = '8K2378DHOQYYR6ZX'
    api_keys['key4'] = 'S1ALL8PICSXLYUHL'
    api_keys['key5'] = '5E1OJV6CRGRYJNJU'
    api_keys['key6'] = 'L8URDZ6Q2NNPCH3R'
    api_keys['key7'] = 'V5XT420902MEY194'
    
    # Get an iterator for the dictionary keys
    keys_iterator = iter(api_keys.keys())
    key = next(keys_iterator)
    status_code =  200

    while status_code == 200:
        value = api_keys[key]
        print(f"Key: {key}, Value: {value}")
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_name}&apikey={value}" ## Daily
        response=requests.get(url)
        #check if the response was successful
        if (response.status_code == 200):
            data=response.json()
            if data is None:
                try:
                # Get the next key from the iterator
                    key = next(keys_iterator)
                # Get the corresponding value
                    value = api_keys[key]
                    print(f"Key: {key}, Value: {value}")
                except StopIteration:
                # Break the loop when all keys have been traversed
                    print ("Out of all AIP keys. Better luck next day!")
                    break
                
            # Traverse the JSON data
            return data
        else:
            print("error")
            print(response.status_code)
            return None

stock_prices = {}
stock_name = input ("Enter a stock name: ""Done"" to finish")
while stock_name.upper() != "DONE":
    data = get_stock_data(stock_name)
    display_stock_data(data)
    stock_name = input ("Enter a stock name: ""Done"" to finish")

Key: key1, Value: 49MPJB381SMB9GL2
Meta Data:
+-------------------+---------------------------------------------------+
|     Meta Data     |                       Values                      |
+-------------------+---------------------------------------------------+
|   1. Information  | Daily Prices (open, high, low, close) and Volumes |
|     2. Symbol     |                        nvda                       |
| 3. Last Refreshed |                     2024-01-18                    |
|   4. Output Size  |                      Compact                      |
|    5. Time Zone   |                     US/Eastern                    |
+-------------------+---------------------------------------------------+

Time Series (Daily):
+------------+----------+----------+----------+----------+-----------+
|    Date    | 1. open  | 2. high  |  3. low  | 4. close | 5. volume |
+------------+----------+----------+----------+----------+-----------+
| 2024-01-18 | 572.6000 | 576.0000 | 561.0700 | 571.0

In [6]:
%pip install prettytable